# WK06: Diffuser Models

### pix2pix / img2img

https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

https://github.com/GaParmar/img2img-turbo

### Scribble

https://huggingface.co/lllyasviel/sd-controlnet-scribble

<img src="./imgs/scribble_00.jpg" width="200px">
<img src="./imgs/scribble_01.jpg" width="200px">

### Depth

https://huggingface.co/lllyasviel/sd-controlnet-depth

<img src="./imgs/depth_00.jpg" width="200px">
<img src="./imgs/depth_01.jpg" width="200px">

In [ ]:
# !pip install diffusers transformers accelerate

In [ ]:
from diffusers import ControlNetModel, UniPCMultistepScheduler
from diffusers import StableDiffusionControlNetPipeline as SDPipeline

from PIL import Image
import torch

In [ ]:
controlnet = ControlNetModel.from_pretrained(
  "lllyasviel/sd-controlnet-scribble",
  # "lllyasviel/sd-controlnet-depth",
  torch_dtype=torch.float16
)

pipe = SDPipeline.from_pretrained(
  "runwayml/stable-diffusion-v1-5",
  controlnet=controlnet,
  safety_checker=None,
  torch_dtype=torch.float16
).to("cuda")

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

In [ ]:
im = Image.open("./imgs/scribble.jpg").convert("RGB")
out = pipe("bag", im, num_inference_steps=20)
display(out["images"][0])

### Image Variation

https://huggingface.co/lambdalabs/sd-image-variations-diffusers

In [ ]:
from diffusers import StableDiffusionImageVariationPipeline as SDPipeline
from torchvision import transforms
from PIL import Image

In [ ]:
sd_pipe = SDPipeline.from_pretrained(
  "lambdalabs/sd-image-variations-diffusers",
  revision="v2.0",
  safety_checker=None
).to("cuda")

In [ ]:
im = Image.open("./imgs/GDTM.jpg").convert("RGB")

preprocessor = transforms.Compose([
  transforms.ToTensor(),
  transforms.Resize(
    (224, 224),
    interpolation=transforms.InterpolationMode.BICUBIC,
    antialias=False),
  transforms.Normalize(
    [0.48145466, 0.4578275, 0.40821073],
    [0.26862954, 0.26130258, 0.27577711]),
])

inp = preprocessor(im).to("cuda").unsqueeze(0)

In [ ]:
out = sd_pipe(inp, guidance_scale=3)
display(out["images"][0])

### Refiner

https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0

In [ ]:
# !pip install diffusers --upgrade
# !pip install invisible_watermark transformers accelerate safetensors

In [ ]:
import torch
from diffusers import StableDiffusionXLImg2ImgPipeline as SDPipeline

In [ ]:
pipe = SDPipeline.from_pretrained(
  "stabilityai/stable-diffusion-xl-refiner-1.0",
  torch_dtype=torch.float16,
  variant="fp16",
  use_safetensors=True,
  add_watermarker=False
).to("cuda")

In [ ]:
im = Image.open("./imgs/GDTM.jpg").convert("RGB")
prompt = "a photo of an astronaut riding a horse on mars"
out = pipe(prompt, image=im)
display(out["images"][0])